In [4]:
import numpy as np
import cv2

from rl_socket import Agent, Environment, Camera

Example of camera's point projection

In [9]:
camera_mat = np.array([[633.0902963939839, 0, 629.0646296262861],
                        [0, 638.754439101168, 362.7401326185789],
                        [0, 0, 1]])
camera_dist = np.array([-0.04797802,  0.04744357,  0.00017416,  0.00067967, -0.00408397])

In [46]:
camera = Camera(camera_mat, camera_dist)
agent = Agent(np.array([0., 0., 0.]), np.array([-np.pi / 2, 0., -np.pi / 2]), camera, np.array([[1, 1], [-1, 1], [0, -1]]), None)

In [51]:
points = np.array([[10, 0, 0]], dtype=np.float32)

In [53]:
points_proj = camera.project_points(points, agent.tvec, agent.rvec)

In [54]:
points_proj

array([[[629.06464, 362.74014]]], dtype=float32)

In [2]:
from scipy.stats import norm

In [7]:
norm.pdf(0) * 100

39.89422804014327